<a href="https://colab.research.google.com/github/ifeyichukwu/AfternoonSessionSEPLP/blob/master/SDG_17_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# SDG17 ML Collaboration - Advanced Modeling Notebook
# This notebook contains advanced machine learning models for SDG analysis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

class SDGMLModeler:
    """
    Advanced ML modeling class for SDG collaboration analysis
    """

    def __init__(self, data_path='data/sdg_processed_data.csv'):
        """Initialize with processed data"""
        self.data_path = data_path
        self.data = None
        self.models = {}
        self.predictions = {}
        self.feature_importance = {}

    def load_processed_data(self):
        """Load the processed SDG data"""
        try:
            self.data = pd.read_csv(self.data_path)
            print(f"✅ Loaded processed data: {self.data.shape}")
            print(f"Columns: {list(self.data.columns)}")
            return self.data
        except Exception as e:
            print(f"❌ Error loading data: {e}")
            return None

    def prepare_features(self):
        """Prepare features for machine learning models"""
        if self.data is None:
            print("❌ No data loaded. Please load data first.")
            return None

        # Create feature matrix
        feature_data = []

        for country in self.data['Country'].unique():
            country_data = self.data[self.data['Country'] == country].sort_values('Year')

            if len(country_data) >= 5:  # Need at least 5 years of data
                for i in range(4, len(country_data)):  # Start from 5th year
                    # Historical features (last 4 years)
                    hist_scores = country_data.iloc[i-4:i]['SDG_Score'].values
                    hist_changes = country_data.iloc[i-3:i]['Score_Change'].values

                    # Calculate trend features
                    trend_slope = np.polyfit(range(4), hist_scores, 1)[0]
                    trend_acceleration = np.polyfit(range(3), hist_changes, 1)[0]
                    score_volatility = np.std(hist_scores)
                    recent_momentum = np.mean(hist_changes[-2:])

                    # Target (next year's score)
                    target_score = country_data.iloc[i]['SDG_Score']
                    current_year = country_data.iloc[i]['Year']

                    feature_row = {
                        'Country': country,
                        'Year': current_year,
                        'Current_Score': hist_scores[-1],
                        'Score_1yr_ago': hist_scores[-2],
                        'Score_2yr_ago': hist_scores[-3],
                        'Score_3yr_ago': hist_scores[-4],
                        'Recent_Change': hist_changes[-1],
                        'Avg_Change_2yr': np.mean(hist_changes[-2:]),
                        'Avg_Change_3yr': np.mean(hist_changes),
                        'Trend_Slope': trend_slope,
                        'Trend_Acceleration': trend_acceleration,
                        'Score_Volatility': score_volatility,
                        'Recent_Momentum': recent_momentum,
                        'Target_Score': target_score
                    }

                    feature_data.append(feature_row)

        self.feature_df = pd.DataFrame(feature_data)
        print(f"✅ Features prepared: {self.feature_df.shape}")
        print(f"Feature columns: {[col for col in self.feature_df.columns if col not in ['Country', 'Year', 'Target_Score']]}")

        return self.feature_df

    def train_prediction_models(self):
        """Train multiple ML models to predict SDG scores"""
        if not hasattr(self, 'feature_df') or self.feature_df is None:
            print("❌ No features prepared. Please run prepare_features() first.")
            return None

        # Prepare training data
        feature_cols = [col for col in self.feature_df.columns
                       if col not in ['Country', 'Year', 'Target_Score']]

        X = self.feature_df[feature_cols]
        y = self.feature_df['Target_Score']

        # Split data (use recent years for testing)
        test_mask = self.feature_df['Year'] >= 2018
        X_train, X_test = X[~test_mask], X[test_mask]
        y_train, y_test = y[~test_mask], y[test_mask]

        print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")

        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Define models to train
        models_to_train = {
            'Linear Regression': LinearRegression(),
            'Ridge Regression': Ridge(alpha=1.0),
            'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
            'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
        }

        # Train and evaluate models
        results = {}

        for name, model in models_to_train.items():
            print(f"\n🤖 Training {name}...")

            # Use scaled data for linear models, original for tree-based
            if 'Regression' in name and 'Random' not in name and 'Gradient' not in name:
                model.fit(X_train_scaled, y_train)
                y_pred = model.predict(X_test_scaled)
            else:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

            # Calculate metrics
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            results[name] = {
                'model': model,
                'predictions': y_pred,
                'mse': mse,
                'rmse': rmse,
                'mae': mae,
                'r2': r2
            }

            print(f"   RMSE: {rmse:.3f}")
            print(f"   MAE: {mae:.3f}")
            print(f"   R²: {r2:.3f}")

            # Store feature importance for tree-based models
            if hasattr(model, 'feature_importances_'):
                importance_df = pd.DataFrame({
                    'feature': feature_cols,
                    'importance': model.feature_importances_
                }).sort_values('importance', ascending=False)

                self.feature_importance[name] = importance_df

        self.models = results
        self.scaler = scaler
        return results

    def create_prediction_visualizations(self):
        """Create visualizations for model predictions"""
        if not self.models:
            print("❌ No models trained. Please run train_prediction_models() first.")
            return None

        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        axes = axes.ravel()

        # Get test data for comparison
        test_mask = self.feature_df['Year'] >= 2018
        y_true = self.feature_df[test_mask]['Target_Score']

        for i, (name, results) in enumerate(self.models.items()):
            y_pred = results['predictions']

            # Prediction vs Actual scatter plot
            axes[i].scatter(y_true, y_pred, alpha=0.6, s=50)
            axes[i].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
            axes[i].set_xlabel('Actual SDG Score')
            axes[i].set_ylabel('Predicted SDG Score')
            axes[i].set_title(f'{name}\nR² = {results["r2"]:.3f}, RMSE = {results["rmse"]:.3f}')
            axes[i].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig('images/prediction_performance.png', dpi=300, bbox_inches='tight')
        plt.show()

        # Feature importance plot for tree-based models
        if self.feature_importance:
            fig, axes = plt.subplots(1, len(self.feature_importance), figsize=(15, 6))
            if len(self.feature_importance) == 1:
                axes = [axes]

            for i, (model_name, importance_df) in enumerate(self.feature_importance.items()):
                top_features = importance_df.head(8)

                axes[i].barh(range(len(top_features)), top_features['importance'])
                axes[i].set_yticks(range(len(top_features)))
                axes[i].set_yticklabels(top_features['feature'])
                axes[i].set_xlabel('Feature Importance')
                axes[i].set_title(f'{model_name}\nTop Features')
                axes[i].invert_yaxis()

            plt.tight_layout()
            plt.savefig('images/feature_importance.png', dpi=300, bbox_inches='tight')
            plt.show()

    def predict_future_scores(self, years_ahead=3):
        """Predict SDG scores for future years"""
        if not self.models:
            print("❌ No models trained. Please run train_prediction_models() first.")
            return None

        # Use the best performing model (highest R²)
        best_model_name = max(self.models.keys(), key=lambda x: self.models[x]['r2'])
        best_model = self.models[best_model_name]['model']

        print(f"🔮 Using {best_model_name} for future predictions (R² = {self.models[best_model_name]['r2']:.3f})")

        # Get latest data for each country
        latest_data = self.data.groupby('Country').tail(5)  # Last 5 years for each country

        future_predictions = []

        for country in self.data['Country'].unique():
            country_data = self.data[self.data['Country'] == country].sort_values('Year').tail(5)

            if len(country_data) >= 4:
                # Start with current data
                current_scores = country_data['SDG_Score'].values
                current_changes = country_data['Score_Change'].values[1:]  # Skip first NaN

                for year_ahead in range(1, years_ahead + 1):
                    # Prepare features for prediction
                    hist_scores = current_scores[-4:]
                    hist_changes = current_changes[-3:] if len(current_changes) >= 3 else current_changes

                    trend_slope = np.polyfit(range(4), hist_scores, 1)[0]
                    trend_acceleration = np.polyfit(range(len(hist_changes)), hist_changes, 1)[0] if len(hist_changes) > 1 else 0
                    score_volatility = np.std(hist_scores)
                    recent_momentum = np.mean(hist_changes[-2:]) if len(hist_changes) >= 2 else hist_changes[-1] if len(hist_changes) > 0 else 0

                    feature_row = np.array([
                        hist_scores[-1],  # Current_Score
                        hist_scores[-2],  # Score_1yr_ago
                        hist_scores[-3],  # Score_2yr_ago
                        hist_scores[-4],  # Score_3yr_ago
                        hist_changes[-1] if len(hist_changes) > 0 else 0,  # Recent_Change
                        np.mean(hist_changes[-2:]) if len(hist_changes) >= 2 else hist_changes[-1] if len(hist_changes) > 0 else 0,  # Avg_Change_2yr
                        np.mean(hist_changes) if len(hist_changes) > 0 else 0,  # Avg_Change_3yr
                        trend_slope,
                        trend_acceleration,
                        score_volatility,
                        recent_momentum
                    ]).reshape(1, -1)

                    # Make prediction
                    if 'Regression' in best_model_name and 'Random' not in best_model_name and 'Gradient' not in best_model_name:
                        predicted_score = best_model.predict(self.scaler.transform(feature_row))[0]
                    else:
                        predicted_score = best_model.predict(feature_row)[0]

                    # Ensure reasonable bounds (SDG scores are typically 40-90)
                    predicted_score = np.clip(predicted_score, 40, 95)

                    future_predictions.append({
                        'Country': country,
                        'Year': 2022 + year_ahead - 1,
                        'Predicted_Score': predicted_score,
                        'Model_Used': best_model_name
                    })

                    # Update arrays for next iteration
                    current_scores = np.append(current_scores[1:], predicted_score)
                    if len(current_changes) > 0:
                        new_change = predicted_score - hist_scores[-1]
                        current_changes = np.append(current_changes[1:], new_change)

        self.future_predictions = pd.DataFrame(future_predictions)
        print(f"✅ Future predictions generated for {len(self.data['Country'].unique())} countries")

        return self.future_predictions

    def visualize_future_predictions(self):
        """Visualize future predictions alongside historical data"""
        if not hasattr(self, 'future_predictions') or self.future_predictions is None:
            print("❌ No future predictions available. Please run predict_future_scores() first.")
            return None

        # Select interesting countries to visualize
        countries_to_plot = ['Germany', 'United States', 'China', 'India', 'Brazil', 'South Africa']

        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.ravel()

        for i, country in enumerate(countries_to_plot):
            if country in self.data['Country'].values:
                # Historical data
                hist_data = self.data[self.data['Country'] == country]

                # Future predictions
                future_data = self.future_predictions[self.future_predictions['Country'] == country]

                # Plot historical
                axes[i].plot(hist_data['Year'], hist_data['SDG_Score'],
                           marker='o', linewidth=2, label='Historical', color='blue')

                # Plot predictions
                if len(future_data) > 0:
                    # Connect last historical point to first prediction
                    last_hist_year = hist_data['Year'].max()
                    last_hist_score = hist_data[hist_data['Year'] == last_hist_year]['SDG_Score'].iloc[0]

                    pred_years = np.concatenate([[last_hist_year], future_data['Year'].values])
                    pred_scores = np.concatenate([[last_hist_score], future_data['Predicted_Score'].values])

                    axes[i].plot(pred_years, pred_scores,
                               marker='s', linewidth=2, linestyle='--', label='Predicted', color='red')

                axes[i].set_title(f'{country}', fontsize=14, fontweight='bold')
                axes[i].set_xlabel('Year')
                axes[i].set_ylabel('SDG Score')
                axes[i].legend()
                axes[i].grid(True, alpha=0.3)
                axes[i].set_xlim(2015, 2025)

        plt.tight_layout()
        plt.savefig('images/future_predictions.png', dpi=300, bbox_inches='tight')
        plt.show()

        # Summary table of predictions
        print("\n📊 FUTURE SDG SCORE PREDICTIONS")
        print("=" * 50)

        latest_predictions = self.future_predictions[self.future_predictions['Year'] == self.future_predictions['Year'].max()]
        latest_predictions_sorted = latest_predictions.sort_values('Predicted_Score', ascending=False)

        print(f"Predictions for {latest_predictions['Year'].iloc[0]}:")
        for _, row in latest_predictions_sorted.iterrows():
            print(f"  {row['Country']}: {row['Predicted_Score']:.1f}")

    def generate_collaboration_recommendations(self):
        """Generate data-driven collaboration recommendations"""
        if not hasattr(self, 'future_predictions'):
            self.predict_future_scores()

        print("\n🤝 COLLABORATION RECOMMENDATIONS")
        print("=" * 50)

        # Get latest historical scores and future predictions
        latest_historical = self.data.groupby('Country')['SDG_Score'].last()
        latest_predictions = self.future_predictions.groupby('Country')['Predicted_Score'].last()

        # Combine data
        collab_data = pd.DataFrame({
            'Current_Score': latest_historical,
            'Predicted_Score': latest_predictions,
            'Predicted_Change': latest_predictions - latest_historical
        }).reset_index()

        # Categorize countries
        high_performers = collab_data[collab_data['Current_Score'] >= 78]['Country'].tolist()
        emerging_strong = collab_data[(collab_data['Current_Score'] >= 70) & (collab_data['Current_Score'] < 78)]['Country'].tolist()
        developing_focus = collab_data[collab_data['Current_Score'] < 70]['Country'].tolist()

        declining_countries = collab_data[collab_data['Predicted_Change'] < -0.5]['Country'].tolist()
        improving_countries = collab_data[collab_data['Predicted_Change'] > 1]['Country'].tolist()

        print("1. 🏆 MENTORSHIP OPPORTUNITIES:")
        print(f"   High Performers: {', '.join(high_performers)}")
        print(f"   Can mentor: {', '.join(developing_focus)}")

        print(f"\n2. 🚀 PEER LEARNING NETWORKS:")
        print(f"   Emerging Strong: {', '.join(emerging_strong)}")
        print("   → Form regional collaboration networks")

        print(f"\n3. ⚠️  PRIORITY INTERVENTION:")
        if declining_countries:
            print(f"   Countries needing support: {', '.join(declining_countries)}")
        else:
            print("   No countries showing significant decline - good news!")

        print(f"\n4. 🌟 SUCCESS STORIES TO SHARE:")
        if improving_countries:
            print(f"   Fast improving countries: {', '.join(improving_countries)}")
            print("   → Study their strategies for replication")

        return collab_data

# Usage example
def main():
    """Main function to demonstrate the ML modeling pipeline"""

    print("🤖 SDG ML MODELING PIPELINE")
    print("=" * 40)

    # Initialize the modeler
    modeler = SDGMLModeler()

    # Load data
    print("\n1. Loading processed data...")
    modeler.load_processed_data()

    # Prepare features
    print("\n2. Preparing ML features...")
    modeler.prepare_features()

    # Train models
    print("\n3. Training prediction models...")
    modeler.train_prediction_models()

    # Create visualizations
    print("\n4. Creating prediction visualizations...")
    modeler.create_prediction_visualizations()

    # Predict future scores
    print("\n5. Predicting future SDG scores...")
    modeler.predict_future_scores(years_ahead=3)

    # Visualize predictions
    print("\n6. Visualizing future predictions...")
    modeler.visualize_future_predictions()

    # Generate recommendations
    print("\n7. Generating collaboration recommendations...")
    recommendations = modeler.generate_collaboration_recommendations()

    print(f"\n✅ ML modeling complete! Check the images folder for visualizations.")

    return modeler

if __name__ == "__main__":
    # Run the complete ML pipeline
    ml_modeler = main()

🤖 SDG ML MODELING PIPELINE

1. Loading processed data...
❌ Error loading data: [Errno 2] No such file or directory: 'data/sdg_processed_data.csv'

2. Preparing ML features...
❌ No data loaded. Please load data first.

3. Training prediction models...
❌ No features prepared. Please run prepare_features() first.

4. Creating prediction visualizations...
❌ No models trained. Please run train_prediction_models() first.

5. Predicting future SDG scores...
❌ No models trained. Please run train_prediction_models() first.

6. Visualizing future predictions...
❌ No future predictions available. Please run predict_future_scores() first.

7. Generating collaboration recommendations...
❌ No models trained. Please run train_prediction_models() first.

🤝 COLLABORATION RECOMMENDATIONS


AttributeError: 'NoneType' object has no attribute 'groupby'